In [ ]:
# by_sina_tijani
# explaining the cleaning process by displaying them all in a single cell 
# the codes were originally written and run in different cells to check the progress.

# my libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# reading the data set directly as the working directory has already been set
omni = pd.read_csv("CW2(2324SepJan)_MarketingCampaignData.csv")

# formatting the columns To lowercase
omni.columns = omni.columns.str.lower()

# Check if the column heads were changed
omni.columns

# Checking for perfect duplicates
omni.duplicated().any()

# isolating the id column to check for duplicates across the other 18 columns
duplicate_check = ['year_birth', 'education', 'marital_status', 'income', 'kidhome',
       'teenhome', 'dt_customer', 'recency', 'mntdrinks', 'mntfruits',
       'mntmeatproducts', 'mntfishproducts', 'mntsweetproducts',
       'numdealspurchases', 'numwebpurchases', 'numapppurchases',
       'numstorepurchases', 'numwebvisitsmonth']

# Checking if there are any duplicates
omni.duplicated(subset=duplicate_check).any()

# printing the number and percent of duplicates
print(f'The dataset has {omni.duplicated(subset=duplicate_check).sum()} near perfect duplicates')
duplicates_percent = len(omni[omni.duplicated(subset=duplicate_check)]) / len(omni) * 100
print(f"And that represents {duplicates_percent:.2f}% of the dataset.")

# Check all the rows with their duplicates
omni[omni.duplicated(subset=duplicate_check, keep=False)].sort_values(by=['year_birth', 'income'], ascending=True)

# Dropping the duplicates while resetting the index
omni2 = omni.drop_duplicates(subset=duplicate_check).reset_index(drop=True)

# changing the data type for dt_customer
omni2.dt_customer = pd.to_datetime(omni2.dt_customer)

# checking if the data type was changed correctly
omni2.info()

# checking for total null values
omni2.isnull().sum()

# finding rows with missing income
missing_income = omni2[omni2.income.isnull()]
missing_income

# filling the missing income with the median of the incomes. and checking if they were filled
omni2.income.fillna(omni2.income.median(), inplace=True)
omni2.isnull().sum()

# finding descriptive statistics to check for outliers presence in the income column
omni2.income.describe().round()

# visualizing the outliers in income column with a boxplot
omni2[['income']].boxplot()
plt.show()

# using interquartile range to find and isolate the outliers
Q1 = omni2.income.quantile(0.25)
Q3 = omni2.income.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1 * IQR
upper_bound = Q3 + 1.5 * IQR

iqr_outliers = omni2[(omni2.income < lower_bound) | (omni2.income > upper_bound)]
iqr_outliers.sort_values(by='income', ascending=True)

# printing the minimum and maximum income based on the IQR calculations
# and printing the count and percentage
print(f'The minimum income of IQR outliers {lower_bound}')
print(f'The maximum income of IQR outliers {upper_bound}\n')

print(f"The number of IQR outliers are {iqr_outliers.income.count()}.")
outlier_percent = len(iqr_outliers) / len(omni2) * 100
print(f"The outliers represents {outlier_percent:.2f}% of the dataset.")

# dropping the outliers from the dataset
omni3 = omni2.drop(iqr_outliers.index)
omni3

# finding descriptive statistics of the cleaned dataset
omni3.describe().round()

# finding mode across 1st part the dataset
omni3[['year_birth', 'education', 'marital_status', 'income', 'kidhome',
       'teenhome', 'dt_customer', 'recency', 'mntdrinks', 'mntfruits', 'mntmeatproducts']].mode()

# finding mode across 2nd part the dataset
omni3[['mntfishproducts', 'mntsweetproducts',
       'numdealspurchases', 'numwebpurchases', 'numapppurchases',
       'numstorepurchases', 'numwebvisitsmonth']].mode()

# checking the latest date the company is operating in
omni3[['dt_customer']].sort_values(by='dt_customer', ascending=False).head()

# exporting cleaned data to excel
omni3.to_excel('cw2_cleaned_dataset.xlsx')

# finding education value counts
omni3.education.value_counts()

# grouping Alone, Absurd and YOLO to a new Other group
omni3.marital_status.value_counts()
group_three = ['Alone', 'Absurd', 'YOLO']
omni3['marital_status_grouped'] = omni3['marital_status'].replace(group_three, 'Other')

# exporting to a different excel file
omni3.to_excel('cw2_cleaned_dataset_gMS.xlsx')

# crosschecking the value count of the new marital status group
omni3.marital_status_grouped.value_counts()



# creating a new educational and marital status columns with their counts
# creating a dictionary to replace the educational categories with scores
edu_nominal_scores = {'Graduation': 1, 'PhD': 2, 'Master': 3, 'Basic': 4, '2n Cycle': 5}
# creating a new column for the education scores
omni3['education_nominal_score'] = omni3.education.replace(edu_nominal_scores)
# creating a dictionary to replace the marital categories with scores
marital_nom_scores = {'Single': 1, 'Together': 2, 'Married': 3, 'Divorced': 4, 
                      'Widow': 5, 'Other': 6}
# creating a new column for the marital status scores
omni3['maritalstatus_nominal_score'] = omni3.marital_status_grouped.replace(marital_nom_scores)
# inspecting if the new columns are added
omni3

# exporting the latest changes to a new excel sheet
omni3.to_excel('cw2_cleaned_dataset_gMS_nsMS_nsE.xlsx')